In [1]:
# bibliotecas
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from pandas_profiling import ProfileReport

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
# dataset
teste = pd.read_csv('teste.csv', decimal='.')
treino = pd.read_csv('treino.csv', decimal='.')

treino.head()

,inadimplente,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
profile = ProfileReport(teste, title="Pandas Profiling Report").to_file('datarisk.html')


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
teste.info()
print('\n')
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   util_linhas_inseguras                  40000 non-null  float64
 1   idade                                  40000 non-null  int64  
 2   vezes_passou_de_30_59_dias             40000 non-null  int64  
 3   razao_debito                           40000 non-null  float64
 4   salario_mensal                         32032 non-null  float64
 5   numero_linhas_crdto_aberto             40000 non-null  int64  
 6   numero_vezes_passou_90_dias            40000 non-null  int64  
 7   numero_emprestimos_imobiliarios        40000 non-null  int64  
 8   numero_de_vezes_que_passou_60_89_dias  40000 non-null  int64  
 9   numero_de_dependentes                  38954 non-null  float64
dtypes: float64(4), int64(6)
memory usage: 3.1 MB


<class 'pandas.core.fra

### Percentual de valores nulos nos datasets

In [4]:
(treino.isnull().sum() / treino.shape[0] * 100).sort_values(ascending=False)

salario_mensal                           19.784545
numero_de_dependentes                     2.616364
inadimplente                              0.000000
util_linhas_inseguras                     0.000000
idade                                     0.000000
vezes_passou_de_30_59_dias                0.000000
razao_debito                              0.000000
numero_linhas_crdto_aberto                0.000000
numero_vezes_passou_90_dias               0.000000
numero_emprestimos_imobiliarios           0.000000
numero_de_vezes_que_passou_60_89_dias     0.000000
dtype: float64

In [5]:
(teste.isnull().sum() / teste.shape[0] * 100).sort_values(ascending=False)

salario_mensal                           19.920
numero_de_dependentes                     2.615
util_linhas_inseguras                     0.000
idade                                     0.000
vezes_passou_de_30_59_dias                0.000
razao_debito                              0.000
numero_linhas_crdto_aberto                0.000
numero_vezes_passou_90_dias               0.000
numero_emprestimos_imobiliarios           0.000
numero_de_vezes_que_passou_60_89_dias     0.000
dtype: float64

In [6]:
teste.nunique().sort_values()

numero_de_vezes_que_passou_60_89_dias       10
numero_de_dependentes                       12
vezes_passou_de_30_59_dias                  14
numero_vezes_passou_90_dias                 18
numero_emprestimos_imobiliarios             21
numero_linhas_crdto_aberto                  52
idade                                       81
salario_mensal                            7879
razao_debito                             33792
util_linhas_inseguras                    34211
dtype: int64

In [7]:
treino.nunique().sort_values()

inadimplente                                 2
numero_de_dependentes                       12
numero_de_vezes_que_passou_60_89_dias       13
vezes_passou_de_30_59_dias                  15
numero_vezes_passou_90_dias                 18
numero_emprestimos_imobiliarios             28
numero_linhas_crdto_aberto                  57
idade                                       86
salario_mensal                           12228
razao_debito                             86002
util_linhas_inseguras                    92671
dtype: int64

##### Os datasets apresentam uma boa relação entre números únicos não sendo necessária ação de ajuste.

##### A Coluna salario_mensal tem 19.92% de valores nulos, sendo salário de grande importância vou utilizar a mediana como referência no preenchimento, desta forma pretendo não influenciar nos resultados mantendo um viés neutro. 

##### A Coluna numero_de_dependentes representa 2.61% dos valores nulos e tem 12 resultados únicos, vou considerar os nulos como não tendo filhos. 

In [8]:
treino.describe()

,inadimplente,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
count,110000.000000,110000.000000,110000.000000,110000.000000,110000.000000,8.823700e+04,110000.000000,110000.000000,110000.000000,110000.000000,107122.000000
mean,0.066645,5.929979,52.255636,0.424055,354.823589,6.637411e+03,8.445573,0.269955,1.019891,0.243891,0.757482
std,0.249408,252.301965,14.768241,4.240780,2074.140421,1.338395e+04,5.139026,4.217326,1.135989,4.204137,1.114670
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.030054,41.000000,0.000000,0.175016,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.155936,52.000000,0.000000,0.366682,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.562806,63.000000,0.000000,0.866874,8.225000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [9]:
print('TREINO')
treino.isnull().sum()

TREINO


inadimplente                                 0
util_linhas_inseguras                        0
idade                                        0
vezes_passou_de_30_59_dias                   0
razao_debito                                 0
salario_mensal                           21763
numero_linhas_crdto_aberto                   0
numero_vezes_passou_90_dias                  0
numero_emprestimos_imobiliarios              0
numero_de_vezes_que_passou_60_89_dias        0
numero_de_dependentes                     2878
dtype: int64

In [10]:
print('TESTE')
teste.isnull().sum()

TESTE


util_linhas_inseguras                       0
idade                                       0
vezes_passou_de_30_59_dias                  0
razao_debito                                0
salario_mensal                           7968
numero_linhas_crdto_aberto                  0
numero_vezes_passou_90_dias                 0
numero_emprestimos_imobiliarios             0
numero_de_vezes_que_passou_60_89_dias       0
numero_de_dependentes                    1046
dtype: int64

In [11]:
median_treino = treino['salario_mensal'].median()
median_teste = teste['salario_mensal'].median()

In [12]:
dic_treino = {'numero_de_dependentes': 0, 'salario_mensal': median_treino}
dic_teste = {'numero_de_dependentes': 0, 'salario_mensal': median_teste}

In [13]:
treino_limpo = treino.fillna(dic_treino)
teste_limpo = teste.fillna(dic_teste)

In [14]:
treino_limpo.isnull().sum()

inadimplente                             0
util_linhas_inseguras                    0
idade                                    0
vezes_passou_de_30_59_dias               0
razao_debito                             0
salario_mensal                           0
numero_linhas_crdto_aberto               0
numero_vezes_passou_90_dias              0
numero_emprestimos_imobiliarios          0
numero_de_vezes_que_passou_60_89_dias    0
numero_de_dependentes                    0
dtype: int64

In [15]:
teste_limpo.isnull().sum()

util_linhas_inseguras                    0
idade                                    0
vezes_passou_de_30_59_dias               0
razao_debito                             0
salario_mensal                           0
numero_linhas_crdto_aberto               0
numero_vezes_passou_90_dias              0
numero_emprestimos_imobiliarios          0
numero_de_vezes_que_passou_60_89_dias    0
numero_de_dependentes                    0
dtype: int64

In [16]:
display(treino_limpo.corr())

,inadimplente,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
inadimplente,1.000000,-0.003263,-0.114281,0.124546,-0.006534,-0.017664,-0.029332,0.116023,-0.008144,0.101536,0.043913
util_linhas_inseguras,-0.003263,1.000000,-0.004444,-0.001336,0.001255,0.007561,-0.010120,-0.001154,0.007590,-0.001105,-0.001467
idade,-0.114281,-0.004444,1.000000,-0.063230,0.025221,0.030098,0.148640,-0.061371,0.032920,-0.057604,-0.214781
vezes_passou_de_30_59_dias,0.124546,-0.001336,-0.063230,1.000000,-0.006860,-0.009015,-0.056347,0.984157,-0.031093,0.987377,-0.005225
razao_debito,-0.006534,0.001255,0.025221,-0.006860,1.000000,-0.018755,0.047158,-0.008344,0.117054,-0.007558,-0.045567
salario_mensal,-0.017664,0.007561,0.030098,-0.009015,-0.018755,1.000000,0.091786,-0.011384,0.124249,-0.010345,0.070212
numero_linhas_crdto_aberto,-0.029332,-0.010120,0.148640,-0.056347,0.047158,0.091786,1.000000,-0.080873,0.433337,-0.071979,0.074057
numero_vezes_passou_90_dias,0.116023,-0.001154,-0.061371,0.984157,-0.008344,-0.011384,-0.080873,1.000000,-0.045902,0.993162,-0.012903
numero_emprestimos_imobiliarios,-0.008144,0.007590,0.032920,-0.031093,0.117054,0.124249,0.433337,-0.045902,1.000000,-0.039987,0.127292
numero_de_vezes_que_passou_60_89_dias,0.101536,-0.001105,-0.057604,0.987377,-0.007558,-0.010345,-0.071979,0.993162,-0.039987,1.000000,-0.013497


In [17]:
x = treino_limpo.drop('inadimplente', axis=1)
y = treino_limpo['inadimplente']

x_teste = teste_limpo

display(x)

,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...
109995,0.137396,59,1,0.448912,9600.0,10,0,2,0,0.0
109996,0.276964,46,0,0.491288,12224.0,19,0,4,0,4.0
109997,0.181257,43,0,0.382635,12000.0,13,0,2,0,2.0
109998,0.037699,86,0,0.248107,7000.0,14,0,2,0,1.0


In [18]:
display(y)

0         1
1         0
2         0
3         0
4         0
         ..
109995    0
109996    0
109997    0
109998    0
109999    0
Name: inadimplente, Length: 110000, dtype: int64

In [19]:
# Árvore de decisão
Arvore = DecisionTreeClassifier()

Arvore.fit(x,y)
print(Arvore.score(x,y))

0.9996909090909091


In [20]:
# Randon Forest
Forest = RandomForestClassifier()

Forest.fit(x,y)
print(Forest.score(x,y))

0.9996454545454545


###### Os dois modelos perfomaram de forma bastante satisfatória demostrando acurácia de 99,96%.
###### Vou utilizar o Randon Forest, pois a tendência é de trabalhar melhor se levarmos em conta a utilização de Bigdata, que acredito ser o escopo do trabalho.

In [21]:
y_pred = Forest.predict(x_teste)
display(y_pred)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [22]:
teste_limpo.insert(loc = 0, column = 'inadimplente', value = y_pred)

In [23]:
display(teste_limpo)

,inadimplente,util_linhas_inseguras,idade,vezes_passou_de_30_59_dias,razao_debito,salario_mensal,numero_linhas_crdto_aberto,numero_vezes_passou_90_dias,numero_emprestimos_imobiliarios,numero_de_vezes_que_passou_60_89_dias,numero_de_dependentes
0,0,0.025849,62,0,0.081775,8180.0,3,0,2,0,0.0
1,0,0.667083,55,0,0.153112,2200.0,3,0,0,0,0.0
2,0,0.007093,44,0,0.148800,7499.0,20,0,1,0,0.0
3,0,0.091213,54,0,0.351635,5900.0,15,0,1,1,0.0
4,0,0.112680,54,0,0.065959,2167.0,3,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
39995,0,0.000000,68,1,0.062858,10833.0,9,1,0,0,0.0
39996,0,0.061117,84,0,0.013598,8456.0,7,0,0,0,0.0
39997,0,0.817745,70,0,0.663056,6000.0,17,0,2,0,0.0
39998,0,0.106139,64,2,0.865438,5045.0,12,0,4,0,0.0


In [24]:
teste_limpo.to_csv('teste_balbinotti.csv')
